# Kaggle New York Stock Exchange Prediction

S&P 500 companies historical prices with fundamental data

* https://www.kaggle.com/dgawlik/nyse

주가 예측은 기본적으로 random walk인 그래프를 예측하는 것이기 때문에 자기 자신의 그래프 또는 과거의 데이터만 가지고서는 예측할수가 없습니다. <br> 
해당 문서에서는 다른 회사의 주식 가격과 더불어 해당 회사의 연도별 회계자료를 통해 예측을 하고자 합니다. 

In [1]:
%pylab inline
import pandas as pd

from IPython.display import display, HTML
from collections import Counter

from keras.models import Sequential
from keras.layers import LSTM, Dense, Activation
from keras.layers.normalization import BatchNormalization
from keras.activations import relu
from keras.metrics import mean_squared_error
from keras.wrappers.scikit_learn import KerasClassifier, KerasRegressor

from sklearn.metrics import make_scorer, r2_score
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler, MinMaxScaler

from IPython.display import SVG, Image
from keras.utils.vis_utils import model_to_dot

pd.set_option('display.max_columns', 500)

Populating the interactive namespace from numpy and matplotlib


Using TensorFlow backend.


In [2]:
CORRELATED_COMPANY_N = 16
COMPANY_SYMBOL = 'AMZN'
BATCH_SIZE=60
TIME_SERIES_LENGTH = 50

# Data 
## securities.csv

기업에 대한 분류및 기타 정보

In [3]:
securities = pd.read_csv('/dataset/stock-market/nyse/securities.csv')

# GICS Sector를 카테고리로 만듬
securities['Sector'] = securities['GICS Sector'].astype('category').cat.codes


display(HTML('<h3>securities.csv</h3>'))
print('Shape:', securities.shape)
display(securities.head())

display(HTML('<h3>예측하려는 회사의 정보</h3>'))
display(securities[securities['Ticker symbol'] == COMPANY_SYMBOL])

Shape: (505, 9)


,Ticker symbol,Security,SEC filings,GICS Sector,GICS Sub Industry,Address of Headquarters,Date first added,CIK,Sector
0,MMM,3M Company,reports,Industrials,Industrial Conglomerates,"St. Paul, Minnesota",NaN,66740,5
1,ABT,Abbott Laboratories,reports,Health Care,Health Care Equipment,"North Chicago, Illinois",1964-03-31,1800,4
2,ABBV,AbbVie,reports,Health Care,Pharmaceuticals,"North Chicago, Illinois",2012-12-31,1551152,4
3,ACN,Accenture plc,reports,Information Technology,IT Consulting & Other Services,"Dublin, Ireland",2011-07-06,1467373,6
4,ATVI,Activision Blizzard,reports,Information Technology,Home Entertainment Software,"Santa Monica, California",2015-08-31,718877,6


,Ticker symbol,Security,SEC filings,GICS Sector,GICS Sub Industry,Address of Headquarters,Date first added,CIK,Sector
26,AMZN,Amazon.com Inc,reports,Consumer Discretionary,Internet & Direct Marketing Retail,"Seattle, Washington",2005-11-18,1018724,0


## fundamentals.csv

년도별 기업의 회계자료

In [ ]:
fundamentals = pd.read_csv('/dataset/stock-market/nyse/fundamentals.csv', index_col=0)
fundamentals['Period Ending'] = pd.to_datetime(fundamentals['Period Ending'])

display(fundamentals.head())
fdm_company = fundamentals[fundamentals['Ticker Symbol'] == COMPANY_SYMBOL]

,Ticker Symbol,Period Ending,Accounts Payable,Accounts Receivable,Add'l income/expense items,After Tax ROE,Capital Expenditures,Capital Surplus,Cash Ratio,Cash and Cash Equivalents,Changes in Inventories,Common Stocks,Cost of Revenue,Current Ratio,Deferred Asset Charges,Deferred Liability Charges,Depreciation,Earnings Before Interest and Tax,Earnings Before Tax,Effect of Exchange Rate,Equity Earnings/Loss Unconsolidated Subsidiary,Fixed Assets,Goodwill,Gross Margin,Gross Profit,Income Tax,Intangible Assets,Interest Expense,Inventory,Investments,Liabilities,Long-Term Debt,Long-Term Investments,Minority Interest,Misc. Stocks,Net Borrowings,Net Cash Flow,Net Cash Flow-Operating,Net Cash Flows-Financing,Net Cash Flows-Investing,Net Income,Net Income Adjustments,Net Income Applicable to Common Shareholders,Net Income-Cont. Operations,Net Receivables,Non-Recurring Items,Operating Income,Operating Margin,Other Assets,Other Current Assets,Other Current Liabilities,Other Equity,Other Financing Activities,Other Investing Activities,Other Liabilities,Other Operating Activities,Other Operating Items,Pre-Tax Margin,Pre-Tax ROE,Profit Margin,Quick Ratio,Research and Development,Retained Earnings,Sale and Purchase of Stock,"Sales, General and Admin.",Short-Term Debt / Current Portion of Long-Term Debt,Short-Term Investments,Total Assets,Total Current Assets,Total Current Liabilities,Total Equity,Total Liabilities,Total Liabilities & Equity,Total Revenue,Treasury Stock,For Year,Earnings Per Share,Estimated Shares Outstanding
0,AAL,2012-12-31,3.068000e+09,-222000000.0,-1.961000e+09,23.0,-1.888000e+09,4.695000e+09,53.0,1.330000e+09,0.0,127000000.0,1.049900e+10,78.0,0.000000e+00,223000000.0,1.001000e+09,-1.813000e+09,-2.445000e+09,0.0,0.0,1.340200e+10,0.000000e+00,58.0,1.435600e+10,-5.690000e+08,8.690000e+08,632000000.0,5.800000e+08,3.060000e+08,4.730000e+08,7.116000e+09,0.0,0.0,0.0,-1.020000e+09,197000000.0,1.285000e+09,4.830000e+08,-1.571000e+09,-1.876000e+09,2.050000e+09,-1.876000e+09,-4.084000e+09,1.124000e+09,3.860000e+08,1.480000e+08,1.0,2.167000e+09,6.260000e+08,4.524000e+09,-2.980000e+09,1.509000e+09,11000000.0,1.514700e+10,-141000000.0,8.450000e+08,10.0,31.0,8.0,72.0,0.0,-9.462000e+09,0.000000e+00,1.297700e+10,1.419000e+09,3.412000e+09,2.351000e+10,7.072000e+09,9.011000e+09,-7.987000e+09,2.489100e+10,1.690400e+10,2.485500e+10,-367000000.0,2012.0,-5.60,3.350000e+08
1,AAL,2013-12-31,4.975000e+09,-93000000.0,-2.723000e+09,67.0,-3.114000e+09,1.059200e+10,75.0,2.175000e+09,0.0,5000000.0,1.101900e+10,104.0,0.000000e+00,935000000.0,1.020000e+09,-1.324000e+09,-2.180000e+09,0.0,0.0,1.925900e+10,4.086000e+09,59.0,1.572400e+10,-3.460000e+08,2.311000e+09,856000000.0,1.012000e+09,-1.181000e+09,-2.350000e+08,1.535300e+10,0.0,0.0,0.0,2.208000e+09,660000000.0,6.750000e+08,3.799000e+09,-3.814000e+09,-1.834000e+09,1.873000e+09,-1.834000e+09,-4.489000e+09,1.560000e+09,5.590000e+08,1.399000e+09,5.0,2.299000e+09,1.465000e+09,7.385000e+09,-2.032000e+09,1.711000e+09,481000000.0,1.491500e+10,-56000000.0,8.530000e+08,8.0,80.0,7.0,96.0,0.0,-1.129600e+10,0.000000e+00,1.291300e+10,1.446000e+09,8.111000e+09,4.227800e+10,1.432300e+10,1.380600e+10,-2.731000e+09,4.500900e+10,4.227800e+10,2.674300e+10,0.0,2013.0,-11.25,1.630222e+08
2,AAL,2014-12-31,4.668000e+09,-160000000.0,-1.500000e+08,143.0,-5.311000e+09,1.513500e+10,60.0,1.768000e+09,0.0,7000000.0,1.562000e+10,88.0,0.000000e+00,829000000.0,1.342000e+09,4.099000e+09,3.212000e+09,0.0,0.0,2.308400e+10,4.091000e+09,63.0,2.703000e+10,3.300000e+08,2.240000e+09,887000000.0,1.004000e+09,1.799000e+09,-1.026000e+09,1.604300e+10,0.0,0.0,0.0,1.700000e+08,-146000000.0,3.080000e+09,-3.150000e+08,-2.911000e+09,2.882000e+09,5.420000e+08,2.882000e+09,2.882000e+09,1.771000e+09,8.000000e+08,4.249000e+09,10.0,2.060000e+09,8.980000e+08,7.059000e+09,-4.559000e+09,8.170000e+08,601000000.0,1.092800e+10,-500000000.0,1.295000e+09,8.0,159.0,7.0,80.0,0.0,-8.562000e+09,-1.052000e+09,2.068600e+10,1.677000e+09,6.309000e+09,4.322500e+10,1.175000e

## prices-split-adjusted.csv

주가 정보

In [ ]:
prices = pd.read_csv('/dataset/stock-market/nyse/prices-split-adjusted.csv')

# Standardization
def stadardization_prices(data):
    # Stadardization
    columns = ['open', 'close', 'low', 'high', 'volume']
    scaler = MinMaxScaler()
    data[columns] = scaler.fit_transform(data[columns])
    return scaler, data

scaler_price, prices = stadardization_prices(prices)
    
# Datetime으로 변환
prices.date = pd.to_datetime(prices.date)

# 재무회계가 존재하는 주가만 남김
prices = prices[prices.symbol.isin(fundamentals['Ticker Symbol'].unique())]

# 주식정보가 1762개를 갖은 데이터만 필터링
def filter_minimum_stock(data, mininum):
    stock_counts = data.groupby('symbol').size()
    clean_symbols = stock_counts[stock_counts >= mininum].index
    data = data[data.symbol.isin(clean_symbols)]
    assert len(clean_symbols) == len(data.groupby('symbol').size())
    return data
prices = filter_minimum_stock(prices, 1762)

# 예측하려는 회사의 주가
company = prices[prices.symbol == COMPANY_SYMBOL]

# 예측하려는 회사와 동일한 시간을 갖은 회사들만 필터링
prices = prices[(prices.date >= company.date.iloc[0]) & 
                (prices.date <= company.date.iloc[-1])]

def test_same_datetime(data, company):
    for s in data.symbol.unique():
        c = data[data.symbol == s]
        assert company.date.iloc[0] == c.date.iloc[0]
        assert company.date.iloc[-1] == c.date.iloc[-1]
test_same_datetime(prices, company)

print('prices shape:', prices.shape)
print('Unique Company:', len(prices['symbol'].unique()))
print('가장많은 일일 데이터:', prices.groupby('symbol').size().max())
display(prices.head())

### Correlation

주가 예측을 하려는 기업과 가장 유사도가 비슷한 기업 그리고 전혀 반대인 기업들을 뽑아냅니다.

In [ ]:
def correlation(data, company):
    open_values = pd.Series(company['open'].values)
    
    corr_companies = []
    for symbol in data.symbol.unique():
        if COMPANY_SYMBOL == symbol:
            continue

        # 4년치 회계정보가 있어야 한다. 
        if fundamentals['Ticker Symbol'].str.contains(symbol).sum() != 4:
            continue

        company2 = prices[prices.symbol == symbol]
    #     corr = r2_score(company['open'], company2['open'])
        corr = open_values.corr(pd.Series(company2['open'].values))

        corr_companies.append((symbol, corr))
    corr_companies = sorted(corr_companies, key=lambda x: -x[1])
    return corr_companies

corr_companies = correlation(prices, company)

selected_companies = []
selected_companies += corr_companies[:int(CORRELATED_COMPANY_N/2)]
selected_companies += corr_companies[-int(CORRELATED_COMPANY_N/2):]

display(selected_companies)

## Training and Test Data

In [ ]:
prices[['open', 'low', 'high', 'volume', 'close']].head()

In [ ]:
def create_stock_data(prices):
    COLUMNS = ['open', 'low', 'high', 'volume', 'close']
    COLUMN_N = len(COLUMNS)
    HALF_BATCH_SIZE = int(BATCH_SIZE/2)
    COMPANY_N = len(company)
    SERIES_N = len(company) - int(BATCH_SIZE/2)  # 1732
    VECTOR_N = len(COLUMNS) * (len(selected_companies) + 1)

    company_names = list(map(lambda x: x[0], selected_companies))
    company_names.insert(0, COMPANY_SYMBOL)
    companies = dict()
    for name in company_names:
        matrix = prices[prices.symbol == name][COLUMNS].as_matrix()
        companies[name] = matrix

    x_data = np.zeros((SERIES_N, VECTOR_N))
    y_data = np.zeros((SERIES_N, 1))

    for i in range(SERIES_N):
        for j, name in enumerate(company_names):
            start = j*COLUMN_N
            end = start+COLUMN_N
            x_data[i, start:end] = companies[name][i]
            y_data[i, :] = companies[name][i + HALF_BATCH_SIZE][-1]
    return x_data, y_data

x_data, y_data = create_stock_data(prices)

In [ ]:
np.random.shuffle(x_data)

## Data Visualization

In [ ]:
def visualize(selected_companies):
    companies = selected_companies.copy()
    companies.insert(0, (COMPANY_SYMBOL, 1.))
    
    f , plots = subplots(len(companies))
    f.set_figheight(len(selected_companies) + 2)
    f.set_figwidth(10)
    for i, (symbol, corr) in enumerate(companies):
        company = prices[prices.symbol == symbol]
        name = securities[securities['Ticker symbol'] == symbol].Security.iloc[0]
        sector = securities[securities['Ticker symbol'] == symbol]['GICS Sector'].values[0]
        p = plots[i]
        line = p.plot(company.close, label=f'[{symbol}] {name} ({sector}) {corr:.5}', color="#555555")[0]
        p.legend(loc='upper left')
        if i == 0:
            line.set_color('red')

visualize(selected_companies)

# Model

In [ ]:
SELECTED_COMPANY_N = len(selected_companies)

def create_model(lstm_out1=5, lstm_out2=5, lstm_out3=5, dense1=2048, dense2=256, dense3=64, dense4=32, 
                 optimizer='adam'):
    model = Sequential(name='lstm')
    model.add(LSTM(lstm_out1, batch_input_shape=(None, None, SELECTED_COMPANY_N), return_sequences=True))
    if lstm_out2 is not None:
        model.add(LSTM(lstm_out2, return_sequences=True))
    
    if lstm_out3 is not None:
        model.add(LSTM(lstm_out3, return_sequences=True))
    
    model.add(Dense(dense1, name='fully_connected_layer01'))
    model.add(BatchNormalization(name='batch01'))
    model.add(Activation('relu'))
    
    model.add(Dense(dense2, name='fully_connected_layer02'))
    model.add(BatchNormalization(name='batch02'))
    model.add(Activation('relu'))
    
    model.add(Dense(dense3, name='fully_connected_layer03'))
    model.add(BatchNormalization(name='batch03'))
    model.add(Activation('relu'))
    
    model.add(Dense(dense4, name='fully_connected_layer04'))
    model.add(BatchNormalization(name='batch04'))
    model.add(Activation('relu'))
    
    model.add(Dense(1, name='fully_connected_layer05'))
    model.compile(loss='mean_squared_error', optimizer=optimizer, metrics=[mean_squared_error])
    return model

model = create_model(lstm_out1=30, lstm_out2=30, lstm_out3=None, dense1=2048, dense2=1024, dense3=256)
SVG(model_to_dot(model, show_shapes=True).create(prog='dot', format='svg'))

# Training

In [ ]:
model.fit()